This sheet is intended to read in data outputs from WEAP and merge them into a single data set.
To do so, the sheet 
the combination of the following columns is unique for each row of data.

Steps to achieve this data merge

-You will need three dataframes, that you loop through to merge datasets
    a. Master dataframe - all data is ultimately added here as new rows
    b. "in progress scenario" dataframe - all data for each scenario is added here as new columns.
    Those columns must match the scenario name, year, and month of existing data in the dataframe.
    c. a "temp" dataframe, which is then added into the "in progress scenario" dataframe
-Read in all csv files for one scenario in sequence. In other words, read in all "CanESM Indoor Conservation" csv files one at a time before moving on to the next scenario.
-For each of these scenarios:
1. Read in a new csv file and create a "temp" dataframe.
2. In the "temp" dataframe, add a new column and populate each row with the name of that scenario (e.g. "CanESM Indoor Conservation")
3. Change the columns of the dataset into something human readable and unique
4. Add the columns of the "temp" dataframe into the "in progress scenario" dataframe, by matching rows with the same scenario name, year, and month.
5. Once all csv files are read in and merged into the "in progress scenario" dataframe, append the rows of that dataframe to the "master" dataframe 

In [1]:
import pandas as pd
import numpy as np

In [2]:
#-You will need three dataframes, that you loop through to merge datasets
#    a. Master dataframe - all data is ultimately added here as new rows
#    b. "in progress scenario" dataframe - all data for each scenario is added here as new columns.
#    Those columns must match the scenario name, year, and month of existing data in the dataframe.
#    c. a "temp" dataframe, which is then added into the "in progress scenario" dataframe

dfMaster = pd.DataFrame()
dfInProgress = pd.DataFrame()
dfTemp = pd.DataFrame()

In [3]:
#create a list for each scenario name
scenarioList = ["CanESM", "CanESM - Max Ag Conservation", "CanESM - Max Wastewater Recycling", 
                "CanESM - Maximum Urban Outdoor Conservation", "CanESM - Maximum Urban Indoor Conservation",
                "CanESM - Max MAR", "CESM1-BGC", "CESM1-BGC - Max Ag Conservation", 
                "CESM1-BGC - Max Wastewater Recycling", "CESM1-BGC - Maximum Urban Outdoor Conservation",
                "CESM1-BGC - Maximum Urban Indoor Conservation", "CESM1-BGC - Max MAR", "CMCC-CM", 
                "CMCC-CM - Max Ag Conservation", "CMCC-CM - Max Wastewater Recycling", 
                "CMCC-CM - Maximum Urban Outdoor Conservation", 
                "CMCC-CM - Maximum Urban Indoor Conservation", "CMCC-CM - Max MAR", "RefLOCA"]

#create a list for each set of data that goes with each scenario name
dataList = ["Groundwater Storage", "Transmission Link Electricity Use Monthly", 
            "Demand Site Electricity Use Monthly", "Electricity Use Return Flows Monthly", 
            "Diversion Electricity Use Monthly", "Supply Delivered Ag Demand Sites Monthly", 
            "Supply Delivered All Indoor Monthly", "Supply Delivered All Outdoor Monthly", 
            "Unmet Demand Ag Demand Monthly", "Unmet Demand All Indoor Demand Monthly",
            "Unmet Demand All Outdoor Demand Monthly", "Coverage All Ag Monthly", 
            "Coverage All Indoor Monthly", "Coverage All Outdoor Monthly", 
            "GW Supply Delivered All Demand Sites", "SW Supply Delivered All Demand Sites", 
            "Hydropower Generation"]

# #check that the lists are working as expected
# print(scenarioList)
# print(dataList)

# #temporarily shorten the scenarioList while developing the code
# print(scenarioList)
# scenarioList=["CanESM", "CanESM - Max Ag Conservation"]
# print(scenarioList)


In [4]:
#clean column names
def column_cleanup(df):
    #Change the columns of the dataset into something human readable and unique
    #just changing year and months column names here
    df = df.rename(columns={"$Columns = Year": "Year", "Timestep": "Month"})
    df.columns = df.columns.str.replace('[Acre-foot]', '[AF]')
    df.columns = df.columns.str.replace('[Gigawatt-Hour]', '[GwH]')
    return df

In [5]:
def change_to_float_dtype(df):
    df.replace(' ', 0, inplace=True)
    all_column_names = df.columns.tolist()

    #loop through the column names to change their data types
    for i in all_column_names:
        #if the column name is scenario, year, or month, change the the appropriate datatype 
        if i not in ("Scenario", "Year", "Month"):
            #for columns that aren't scenario, year, and month, change the datatype to float
            df[i] = df[i].astype(float)
    return df

In [6]:
def add_groundwater_storage_total(df):
    df = change_to_float_dtype(df)
    
    #create a list of columns that are supply delivery values
    gw_storage_columns = [col for col in df.columns if col not in['Scenario', 'Year', 'Month']]
    
    df['Total_gw_storage'] = df[gw_storage_columns].sum(axis=1)

    return df

In [7]:
def add_electric_demand_total(df):
    #create a list of columns that are electricity values
    electricity_columns = [col for col in df.columns if '[GwH]' in col]
    # create and fill a new column that sums the electricity use across each row of data
    df['Total_GwH'] = df[electricity_columns].sum(axis=1)
    return df

In [8]:
#add a total for supply deliveries
def add_supply_del_total(df, key_string, dataString):
    df = change_to_float_dtype(df)
    
    #create a list of columns that are supply delivery values
    supply_delivered_total = [col for col in df.columns if key_string in col]
    
    # create and fill a new column that sums the supply delivery values across each row of data
    if 'Ag ' in dataString:
        df['Supply_Del_Ag_Total[AF]'] = df[supply_delivered_total].sum(axis=1)
    elif 'Indoor ' in dataString:
        df['Supply_Del_Indoor_Total[AF]'] = df[supply_delivered_total].sum(axis=1)
    else:
        df['Supply_Del_Outdoor_Total[AF]'] = df[supply_delivered_total].sum(axis=1)

    return df


# dfTemp = column_cleanup(dfTemp)
# dfTemp

In [9]:
def add_unmet_demand_total(df, key_string, dataString):
    df = change_to_float_dtype(df)
    
    #create a list of columns that are supply delivery values
    unmet_demand_total = [col for col in df.columns if key_string in col]
    
    # create and fill a new column that sums the supply delivery values across each row of data
    if 'Ag ' in dataString:
        df['Unmet_Demand_Ag_Total[AF]'] = df[unmet_demand_total].sum(axis=1)
    elif 'Indoor ' in dataString:
        df['Unmet_Demand_Indoor_Total[AF]'] = df[unmet_demand_total].sum(axis=1)
    else:
        df['Unmet_Demand_Outdoor_Total[AF]'] = df[unmet_demand_total].sum(axis=1)

    return df

In [10]:
def add_hydro_generation_total(df):
    df = change_to_float_dtype(df)
    #create a list of columns that are electricity values
    generation_columns = [col for col in df.columns if 'Hydro_' in col]
    # create and fill a new column that sums the electricity use across each row of data
    df['Total_hydro_generation'] = df[generation_columns].sum(axis=1)
    return df

In [11]:
def prepare_columns(df, dataString):
    #cleanup the column names
    df = column_cleanup(df)

    #if the year column is wrong, fix it
    if df['Year'].iloc[0] == 0:
        df['Year'] += 2010
    
    if dataString == '_Groundwater Storage':
        df = df.rename(columns = lambda x: f"GW_storage_{x}" if x not in {"Scenario", "Year", "Month"} else x)
        df = add_groundwater_storage_total(df)
        return df
    
    if 'Electricity' in dataString:
        df = df.rename(columns = lambda x: f"Elec_{x}" if x not in {"Scenario", "Year", "Month"} else x)
        return df
    
    #add prefixes to supply delivered columns
    if '_Supply Delivered' in dataString:
        df = df.rename(columns = lambda x: f"Supply_Del_{x}" if x not in {"Scenario", "Year", "Month"} else x)
        df = add_supply_del_total(df, 'Supply_Del', dataString)
        return df
    
    #add prefixes to unmet demand columns
    if '_Unmet Demand' in dataString:
        df = df.rename(columns = lambda x: f"Unmet_Dem_{x}" if x not in {"Scenario", "Year", "Month"} else x)
        df = add_unmet_demand_total(df, 'Unmet_Dem', dataString)
        return df

    #add prefixes to coverage columns
    if '_Coverage' in dataString:
        df = df.rename(columns = lambda x: f"Coverage_{x}" if x not in {"Scenario", "Year", "Month"} else x)
        return df
    
    #add prefixes to SW and GW Deliveries columns
    if '_GW' in dataString:
        df = df.rename(columns = lambda x: f"GW_{x}" if x not in {"Scenario", "Year", "Month"} else x)
        return df
    
    if '_SW' in dataString:
        df = df.rename(columns = lambda x: f"SW_{x}" if x not in {"Scenario", "Year", "Month"} else x)
        return df
    
    if dataString == '_Hydropower Generation':
        df = df.rename(columns = lambda x: f"Hydro_{x}" if x not in {"Scenario", "Year", "Month"} else x)
        df = add_hydro_generation_total(df)
        return df

    return df

In [12]:
#-Read in all csv files for one scenario in sequence.
#In other words, read in all "CanESM Indoor Conservation" csv files one at a time 
#before moving on to the next scenario.
# -For each of these scenarios:
# 1. Read in a new csv file and put it into the "dfTemp" dataframe

for scenario in scenarioList:
    for data in dataList:
        fileName = scenario + "_" + data +".csv"
        dfTemp = pd.read_csv(fileName, skiprows=5)
        
        #2. In the "temp" dataframe, add a new column and populate each row with the name of that scenario (e.g. "CanESM Indoor Conservation")
        dfTemp.insert(0, "Scenario", scenario, False)
        
        #set dataString
        dataString = '_' + data

        #call function that cleans, prepares, and adds columns needed for analysis
        dfTemp = prepare_columns(dfTemp, dataString)

        #Add the columns of the "temp" dataframe into the "in progress scenario" dataframe
        #by matching rows with the same scenario name, year, and month.
        if dfInProgress.empty == True:
            dfInProgress = dfTemp.copy()
        else:
            dfInProgress = pd.merge(left=dfInProgress, right=dfTemp, on=["Scenario", "Year", "Month"], how = "outer")
            
            #if we are on the final electricity demand column
            #add a column that totals electricity demand in the df
            if data == 'Diversion Electricity Use Monthly':
                dfInProgress = add_electric_demand_total(dfInProgress)

    #once all columns have been added for a scenario
    #if dfMaster is empty, copye dfInProgress into it
    if dfMaster.empty == True:
        dfMaster = dfInProgress.copy()
    else:
        #else, merge the two by adding dfInProgress's rows at the bottom
        dfMaster = pd.concat([dfMaster, dfInProgress], ignore_index=True)
    
    dfInProgress.drop(dfInProgress.index, inplace=True)

# dfMaster["Year"] = dfMaster["Year"] + 2010
# dfMaster

C:\Users\Sunny\AppData\Local\Temp\ipykernel_3736\1642769706.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Supply_Del_Ag_Total[AF]'] = df[supply_delivered_total].sum(axis=1)
C:\Users\Sunny\AppData\Local\Temp\ipykernel_3736\4003975247.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Unmet_Demand_Ag_Total[AF]'] = df[unmet_demand_total].sum(axis=1)
C:\Users\Sunny\AppData\Local\Temp\ipykernel_3736\996053116.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

In [13]:
dfMaster

,Scenario,Year,Month,GW_storage_AZ_APS_E_GW[AF],GW_storage_AZ_APS_N_GW[AF],GW_storage_AZ_APS_SW_GW[AF],GW_storage_AZ_NM_N_GW[AF],GW_storage_AZ_NW_GW[AF],GW_storage_AZ_PHX_GW[AF],GW_storage_AZ_SE_GW[AF],...,Hydro_W E Warne[GwH],Hydro_Wanapum[GwH],Hydro_Wells[GwH],Hydro_Whiskey Town[GwH],Hydro_White Rock_Slab Creek[GwH],Hydro_Woodleaf[GwH],Hydro_Yale[GwH],Hydro_Yellowtail[GwH],Hydro_Yelm[GwH],Total_hydro_generation
0,CanESM,2010,1,14821924.0,14250537.0,12135568.0,14831885.0,14811264.0,13726903.0,11529039.0,...,0.154200,492.072,280.113,0.140153,28.6522,0.0,8.23289,4.59806,0.0,7924.995725
1,CanESM,2010,2,14825600.0,14264701.0,12210586.0,14863986.0,14832764.0,13763142.0,11631681.0,...,22.144800,364.072,208.027,0.233827,43.6554,0.0,11.57630,7.43324,0.0,7436.113070
2,CanESM,2010,3,14833201.0,14279168.0,12278781.0,14895719.0,14854093.0,13800723.0,11729431.0,...,24.888900,429.254,245.877,0.492617,83.6972,0.0,19.15160,12.50270,0.0,10295.625882
3,CanESM,2010,4,14843047.0,14290362.0,12332441.0,14930251.0,14865598.0,13812738.0,11777968.0,...,3.856820,450.869,271.077,0.776172,97.6122,0.0,25.23580,18.58560,0.0,11858.278654
4,CanESM,2010,5,14847775.0,14289094.0,12358096.0,14954027.0,14866241.0,13798403.0,11795369.0,...,0.009754,528.715,311.237,1.197630,96.6157,0.0,36.46130,33.75360,0.0,15067.588902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13903,RefLOCA,2070,8,15114547.0,14261173.0,14492672.0,14838249.0,14988722.0,13745512.0,13879380.0,...,40.490900,439.724,267.664,2.057830,65.8910,0.0,46.55230,54.53560,0.0,15603.741759
13904,RefLOCA,2070,9,15107157.0,14268942.0,14502451.0,14841304.0,14986821.0,13790707.0,13875676.0,...,29.837900,288.759,168.670,1.454000,53.1022,0.0,34.88080,43.28540,0.0,11790.888032
13905,RefLOCA,2070,10,15097183.0,14260489.0,14506288.0,14841183.0,14983413.0,13755930.0,13865853.0,...,26.374800,319.323,189.855,1.452390,41.7506,0.0,40.46080,41.26220,0.0,12360.581668
13906,RefLOCA,2070,11,15089849.0,14269513.0,14516482.0,14839820.0,14981467.0,13800517.0,13889556.0,...,14.020200,374.880,213.850,1.281850,28.8599,0.0,64.49700,39.93080,0.0,13662.980693


In [41]:
dfMaster.to_csv('Combined Data.csv')

In [ ]:
# scenario = 'CanESM'
# data ='Supply Delivered Ag Demand Sites Monthly'

# fileName = scenario + "_" + data +".csv"
# dfTemp = pd.read_csv(fileName, skiprows=5)
        
# #2. In the "temp" dataframe, add a new column and populate each row with the name of that scenario (e.g. "CanESM Indoor Conservation")
# dfTemp.insert(0, "Scenario", scenario, False)
        
# #3. Change the columns of the dataset into something human readable and unique
# #just changing year and months column names here
# dfTemp = dfTemp.rename(columns={"$Columns = Year": "Year", "Timestep": "Month"})
# dfTemp = column_cleanup(dfTemp)

# if 'Supply Delivered' in data:
#     dfTemp = dfTemp.rename(columns = lambda x: f"Supply_Del_{x}" if x not in {"Scenario", "Year", "Month"} else x)

# dfTemp